<a href="https://colab.research.google.com/github/nycolexavier/movies_analyze/blob/main/Movie_Impacta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Contexto

Esta base de dados consiste de filmes lançados até julho de 2017. Os dados incluem elenco, equipe, gênero, orçamento, receita, datas de lançamento, idiomas, empresas de produção, países, contagem de votos e média de votos.


# Movies dataset



## Análise exploratória dos dados

O dataset que iremos trabalhar possui os seguintes (principais) atributos:

* **budget** - O orçamento em que o filme foi feito
* **genre** - O gênero do filme, ação, comédia, suspense etc.
* **homepage** - Um link para a página inicial do filme.
* **id** - Este é, de fato, o movie_id como no primeiro conjunto de dados.
* **keywords** - As palavras-chave ou tags relacionadas ao filme.
* **original_language** - O idioma em que o filme foi feito.
* **original_title** - O título do filme antes da tradução ou adaptação.
* **overview** - Uma breve descrição do filme.
* **popularity** - Uma quantidade numérica que especifica a popularidade do filme.
* **production_companies** -A casa de produção do filme.
* **production_countries** -O país em que foi produzido.
* **release_date** - A data em que foi lançado.
* **revenue** - A receita mundial gerada pelo filme.
* **runtime** - O tempo de execução do filme em minutos.
* **status** - "Lançado" ou "Rumor".
* **tagline** - Slogan do filme.
* **title** - Título do filme.
* **vote_average** - classificações médias que o filme recebeu.
* **vote_count** - a contagem dos votos recebidos.

In [10]:
"""
Na primeira execução deste notebook,
atualize a versão dos seguintes pacontes, é só descomentar as seguintes linhas
e executar a célula.

Após a atualização, lembre-se de reiniciar o Kernel do notebook.
"""
!pip install --upgrade numpy
!pip install --upgrade pandas
!pip install --upgrade matplotlib

Requirement already up-to-date: numpy in /usr/local/lib/python3.7/dist-packages (1.21.0)
Requirement already up-to-date: pandas in /usr/local/lib/python3.7/dist-packages (1.3.0)
Requirement already up-to-date: matplotlib in /usr/local/lib/python3.7/dist-packages (3.4.2)


In [11]:
# importando as bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Leitura e limpeza dos dados

In [12]:
# leitura dos dados
df = pd.read_csv("https://pycourse.s3.amazonaws.com/movies.csv")
df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


AttributeError: ignored

   adult  ... vote_count
0  False  ...     5415.0
1  False  ...     2413.0
2  False  ...       92.0
3  False  ...       34.0
4  False  ...      173.0

[5 rows x 24 columns]

In [13]:
"""
DICA: pandas profile https://github.com/pandas-profiling/pandas-profiling
Para instalar, descomente e execute as seguintes linhas.
"""
!pip install pandas-profiling
!pip install --upgrade pandas-profiling

  Using cached https://files.pythonhosted.org/packages/3b/a3/34519d16e5ebe69bad30c5526deea2c3912634ced7f9b5e6e0bb9dbbd567/pandas_profiling-3.0.0-py2.py3-none-any.whl
     |████████████████████████████████| 3.1MB 3.7MB/s 
     |████████████████████████████████| 81kB 8.6MB/s 
  Using cached https://files.pythonhosted.org/packages/80/96/01e4ba22cef96ae5035dbcf0451c2f4f859f8f17393b98406b23f0034279/visions-0.7.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/7a/a5/393c087efdc78091afa2af9f1378762f9821c9c1d7a22c5753fb5ac5f97a/PyYAML-5.4.1-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/b7/ce/193e8ddf62d4be643b9b4b20e8e9c63b2f6a20f92778c0410c629f89bdaa/phik-0.11.2.tar.gz
  Using cached https://files.pythonhosted.org/packages/29/c1/24814557f1d22c56d50280771a17307e6bf87b70727d975fd6b2ce6b014a/requests-2.25.1-py2.py3-none-any.whl
     |████████████████████████████████| 10.1MB 18.5MB/s 
     |████████████████████████████████| 296kB 3

In [14]:
# descritivo dos dados
from pandas_profiling import ProfileReport

profile = ProfileReport(df, title='Movies dataset')

OSError: ignored

In [ ]:
profile

In [ ]:
# info
df.info()

In [ ]:
# limpeza: remoção de colunas com poucas entradas válidas
df.drop(['belongs_to_collection',
         'homepage',
         'tagline'],
         axis='columns', inplace=True)    
        

In [ ]:
# limpeza: remoção de colunas com pouca variabilidade ou
# irrelevante para a análise
df.drop(['adult', 'overview'],
        axis='columns',
        inplace=True)

In [ ]:
# info
df.info()

In [ ]:
 # limpeza: remoção de linhas com dados faltantes
 df.dropna(axis='index', inplace=True)

In [ ]:
# info
df.info()

In [ ]:
# dados filtrados
df.head()

## Estruturando os dados

Alguns atributos apresentam a seguinte estrutura: `[{'id': id, 'name':name}]`.

Precisamos definir uma estrutura mais simples para análise...

In [ ]:
# estrutura original
df[['genres',
    'production_countries',
    'spoken_languages']].head()

Iremos codificar esses atributos através da operação de one-hot-encoding:

![ohe-hot-enconding](https://pycourse.s3.amazonaws.com/ohe.png)

In [ ]:
import json

def list_to_ohe(df: pd.DataFrame, cols: list):

  # para cada coluna
  n_rows = df.shape[0]
  df.reset_index(inplace=True, drop=True)
  for col_i in cols:

    # selecionado a coluna
    dfi = df[col_i]

    # dicionário para mapeamento
    new_cols = {}

    # percorrendo cada linha do dataframe
    for i, row in enumerate(dfi):

      # leitura da string como um JSON
      row = row.replace("\'", "\"")
      list_i = json.loads(row)

      # percorrendo cada elemento da lista
      for elem in list_i:
      
        # nova coluna com a categoria
        new_col_name = col_i + '_' + elem['name']  # col_[nome_categoria]
    
        # adiciona nova coluna
        if new_col_name not in new_cols:
          new_cols[new_col_name] = np.zeros((n_rows))
        
        # atribui classificação
        new_cols[new_col_name][i] = 1
    
    # append new columns
    new_df = pd.DataFrame(new_cols)
    df = pd.concat([df, new_df],
                   axis=1).reset_index(drop=True) 
  
  return df

In [ ]:
# colunas para transformar
cols_to_transform = ['genres']

print('Shape antes:', df.shape)
df = list_to_ohe(df, cols=cols_to_transform)
print('Shape depois:', df.shape)


In [ ]:
# verificando novas colunas de generos de filmes
genres_attr = [col for col in df if col.startswith('genres_')]
print("Colunas de gênero de filme inseridas:\n",
      np.array(genres_attr).reshape(-1, 1))

In [ ]:
# dataframe tratado
df.head()

## Análise

### Definindo um score

Precisamos definir um score para comparar avaliações entre filmes, para levar em conta a quantidade de avaliações que cada filme recebeu. Para isso, será utilizada a fórmula do IMDB para definição do score de um filme:

$$\mathrm{Weighted~Rating~(WR)} = \frac{v}{v+m}R + \frac{m}{v+m}C,$$

onde:

- $v$ é o número de avaliações (**vote_count**);
- $m$ é o número mínimo de avaliações necessárias para contabilização;
- $R$ é a nota média do filme (**vote_average**);
- $C$ é a média de todas as notas.

A variável $C$ pode ser calculada da seguinte maneira:


In [ ]:
# quantidade de filmes sem avaliação
cond = df['vote_count'] < 1e-3 # 0.001
print("Quantidade de filmes sem avaliação:", sum(cond))

In [ ]:
# removendo da análise filmes que não receberam avaliações
df = df.loc[~cond]
print("Novo shape:", df.shape)

In [ ]:
# verificando operação
cond = df['vote_count'] < 1e-3
print("Quantidade de filmes sem avaliação:", sum(cond))

In [ ]:
# C: nota média entre todos os filmes
C = df['vote_average'].mean()
print("Média de todas as notas (C):", C)

O número mínimo de votos pode ser obtido a partir dos percentis de **vote_count**. Iremos levar em consideração na análise somente os filmes que receberam mais votos que pelo menos 90\%  dos filmes da lista.

In [ ]:
# estatísticas dos votos
df[['vote_count']].describe()

In [ ]:
# m: número mínimo de votos para análise
m = df['vote_count'].quantile(0.9)
print("Número mínimo de votos (m):", m)

In [ ]:
# filtrando o dataset
df = df[df['vote_count']>m]

Cálculo do score:

In [ ]:
# adição da nova coluna com o score
v = df['vote_count']
R = df['vote_average']
df.loc[:, 'score'] = v/(v+m) * R + m/(m+v) * C

In [ ]:
# ordenando o dataframe pelo score calculado
df.sort_values(by='score',
               ascending=False,
               inplace=True)
df.reset_index(inplace=True)

### TOP 10: score

In [ ]:
# TOP 10: score
df[['original_title', 'vote_count', 'vote_average', 'score']].head(10)

In [ ]:
# gênero de filmes no TOP 10 de score
df_top_score = df[:10]

# contagem de gêneros
df_top_score_gen = df_top_score[genres_attr].sum().sort_values(ascending=False)
df_top_score_gen

In [ ]:
# color map
from matplotlib import cm, rcParams
cmap = cm.get_cmap('Set3')
rcParams['figure.figsize'] = (7, 7) 

In [ ]:
# retirando valores nulos
df_top_score_gen = df_top_score_gen[df_top_score_gen > 0]

# formatação dos nomes para visualização
labels = [gen[7:] for gen in df_top_score_gen.index]

# plot
df_top_score_gen.plot.pie(autopct='%1.1f%%',
                          pctdistance=0.8,
                          radius=1.25,
                          labels=labels,
                          cmap=cmap)
plt.ylabel(' ')
plt.suptitle('Gêneros mais frequentes no TOP 10 (score)');

### TOP 10: popularidade

In [ ]:
# ordenando o dataframe pela popularidade
df['popularity'] = df['popularity'].astype(float) 
df_top_pop = df.sort_values(by='popularity',
                            ascending=False)

In [ ]:
# TOP 10: popularidade
df_top_pop[['original_title', 'score', 'popularity']].head(10)

In [ ]:
# visualização dos filmes mais populares
df_top_pop[:10].plot.barh(x='original_title',
                          y=['popularity'])
plt.gca().invert_yaxis();

In [ ]:
# gênero de filmes no TOP 10 de popularity
df_top_pop_gen = df_top_pop[genres_attr]
df_top_pop_gen = df_top_pop_gen[:10].sum().sort_values(ascending=False)

# filtro em valores maiores que 0
df_top_pop_gen = df_top_pop_gen[df_top_pop_gen > 0]
df_top_pop_gen

In [ ]:
# formatação dos nomes para visualização
labels = [gen[7:] for gen in df_top_pop_gen.index]

# plot
df_top_pop_gen.plot.pie(autopct='%1.1f%%',
                          pctdistance=0.8,
                          radius=1.25,
                          labels=labels,
                          cmap=cmap)
plt.ylabel(' ')
plt.suptitle('Gêneros mais frequentes no TOP 10 (popularidade)');

### TOP 10: receita

In [ ]:
# ordenando o dataframe pela receita
df['revenue'] = df['revenue']/1e6  # em milhões de dólares
df_top_rev = df.sort_values(by='revenue',
                            ascending=False)

In [ ]:
# TOP 10: receita
df_top_rev[['original_title', 'score', 'popularity', 'revenue']].head(10)

In [ ]:
# gênero de filmes no TOP 10 de revenue
df_top_rev_gen = df_top_rev[genres_attr]
df_top_rev_gen = df_top_rev_gen[:10].sum().sort_values(ascending=False)

# filtro em valores maiores que 0
df_top_rev_gen = df_top_rev_gen[df_top_rev_gen > 0]
df_top_rev_gen

In [ ]:
# formatação dos nomes para visualização
labels = [gen[7:] for gen in df_top_rev_gen.index]

# plot
df_top_rev_gen.plot.pie(autopct='%1.1f%%',
                        pctdistance=0.8,
                        radius=1.25,
                        labels=labels,
                        cmap=cmap)
plt.ylabel(' ')
plt.suptitle('Gêneros mais frequentes no TOP 10 (receita)');